In [ ]:
# initialization
import pandas as pd
import pyspark.sql.functions
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.types import StringType,DecimalType
from pyspark.sql.functions import input_file_name, substring
import matplotlib.pyplot as plt

## Extract

In [ ]:
# read downloaded csv
raw_sahie = pd.read_csv('/dbfs/mnt/capstone-group2-data/datain/rawdata/sahie_2019.csv', skiprows = 79)
raw_sahie.info()

/databricks/python/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
 has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320298 entries, 0 to 320297
Data columns (total 26 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 year 320298 non-null int64 
 1 version 320298 non-null object 
 2 statefips 320298 non-null int64 
 3 countyfips 320298 non-null int64 
 4 geocat 320298 non-null int64 
 5 agecat 320298 non-null int64 
 6 racecat 320298 non-null int64 
 7 sexcat 320298 non-null int64 
 8 iprcat 320298 non-null int64 
 9 NIPR 320298 non-null object 
 10 nipr_moe 320298 non-null object 
 11 NUI 320298 non-null object 
 12 nui_moe 320298 non-null object 
 13 NIC 320298 non-null object 
 14 nic_moe 320298 non-null object 
 15 PCTUI 320298 non-null object 
 16 pctui_moe 320298 non-null object 
 17 PCTIC 320298 non-null object 
 18 pctic_moe 320298 non-null object 
 19 PCTELIG 320298 non-null object 
 20 pctelig_moe 320298 non-null object 
 21 PCTLIIC 320298 non-null object 
 22 pctliic_moe 320298 non-null object 
 23 state_name 320298 non-null object 
 24 county_name 320298 non-null object 
 25 Unnamed: 25 0 non-null float64
dtypes: float64(1), int64(8), object(17)
memory usage: 63.5+ MB

## Transform

In [ ]:
# operation done on pandas dataframe

# dropping year, version, non-percentage, and margin of error (MOE) columns
# reorganizing columns
raw_sahie = raw_sahie[['geocat','state_name','county_name','agecat','racecat','sexcat','iprcat','PCTUI','PCTIC','PCTELIG','PCTLIIC']]

# stripping white space from state_name and county_name
raw_sahie['state_name'] = raw_sahie['state_name'].str.strip()
raw_sahie['county_name'] = raw_sahie['county_name'].str.strip()

# replacing variables with descriptions
# this will help with sql joins and improve readability
raw_sahie['geocat'] = raw_sahie['geocat'].replace(to_replace=40, value='State geographic identifier')
raw_sahie['geocat'] = raw_sahie['geocat'].replace(to_replace=50, value='County geographic identifier')

raw_sahie['agecat'] = raw_sahie['agecat'].replace(to_replace=0, value='Under 65 years')
raw_sahie['agecat'] = raw_sahie['agecat'].replace(to_replace=1, value='18 to 64 years')
raw_sahie['agecat'] = raw_sahie['agecat'].replace(to_replace=2, value='40 to 64 years')
raw_sahie['agecat'] = raw_sahie['agecat'].replace(to_replace=3, value='50 to 64 years')
raw_sahie['agecat'] = raw_sahie['agecat'].replace(to_replace=4, value='Under 19 years')
raw_sahie['agecat'] = raw_sahie['agecat'].replace(to_replace=5, value='21 to 64 years')

raw_sahie['racecat'] = raw_sahie['racecat'].replace(to_replace=0, value='All races')
raw_sahie['racecat'] = raw_sahie['racecat'].replace(to_replace=1, value='Non-Hispanic white, single race')
raw_sahie['racecat'] = raw_sahie['racecat'].replace(to_replace=2, value='Non-Hispanic black, single race')
raw_sahie['racecat'] = raw_sahie['racecat'].replace(to_replace=3, value='Hispanic or Latino, any race')

raw_sahie['sexcat'] = raw_sahie['sexcat'].replace(to_replace=0, value='both sexes')
raw_sahie['sexcat'] = raw_sahie['sexcat'].replace(to_replace=1, value='male')
raw_sahie['sexcat'] = raw_sahie['sexcat'].replace(to_replace=2, value='female')

raw_sahie['iprcat'] = raw_sahie['iprcat'].replace(to_replace=0, value='All income levels')
raw_sahie['iprcat'] = raw_sahie['iprcat'].replace(to_replace=1, value='At or below 200% of poverty')
raw_sahie['iprcat'] = raw_sahie['iprcat'].replace(to_replace=2, value='At or below 250% of poverty')
raw_sahie['iprcat'] = raw_sahie['iprcat'].replace(to_replace=3, value='At or below 138% of poverty')
raw_sahie['iprcat'] = raw_sahie['iprcat'].replace(to_replace=4, value='At or below 400% of poverty')
raw_sahie['iprcat'] = raw_sahie['iprcat'].replace(to_replace=5, value='Between 138% - 400% of poverty')

# renaming columns
cleaned_sahie = raw_sahie.rename(columns = {
    'geocat':'Geography_Category',
    'state_name':'State_Name',#
    'county_name':'County_Name',
    'agecat':'Age_Category',#
    'racecat':'Race_Category',#
    'sexcat':'Sex_Category',#
    'iprcat':'Income_Category',
    'PCTUI':'Percent_of_Demographic_Uninsured_by_Income_Category',
    'PCTIC':'Percent_of_Demographic_Insured_by_Income_Category',
    'PCTELIG':'Total_Percent_of_Demographic_Uninsured',
    'PCTLIIC':'Total_Percent_of_Demographic_Insured'
})

# deleting blanks 
cleaned_sahie = cleaned_sahie[cleaned_sahie['Percent_of_Demographic_Insured_by_Income_Category'] != '   . ']

# changing types
cleaned_sahie = cleaned_sahie.astype({
    'Percent_of_Demographic_Uninsured_by_Income_Category':'float',
    'Percent_of_Demographic_Insured_by_Income_Category':'float',
    'Total_Percent_of_Demographic_Uninsured':'float',
    'Total_Percent_of_Demographic_Insured':'float'
    
})
cleaned_sahie.head()

Out[4]:

,Geography_Category,State_Name,County_Name,Age_Category,Race_Category,Sex_Category,Income_Category,Percent_of_Demographic_Uninsured_by_Income_Category,Percent_of_Demographic_Insured_by_Income_Category,Total_Percent_of_Demographic_Uninsured,Total_Percent_of_Demographic_Insured
0,State geographic identifier,Alabama,,Under 65 years,All races,both sexes,All income levels,11.6,88.4,11.6,88.4
1,State geographic identifier,Alabama,,Under 65 years,All races,both sexes,At or below 200% of poverty,18.9,81.1,6.8,29.0
2,State geographic identifier,Alabama,,Under 65 years,All races,both sexes,At or below 250% of poverty,18.0,82.0,8.0,36.3
3,State geographic identifier,Alabama,,Under 65 years,All races,both sexes,At or below 138% of poverty,19.9,80.1,4.8,19.1
4,State geographic identifier,Alabama,,Under 65 years,All races,both sexes,At or below 400% of poverty,15.1,84.9,10.1,56.7


## Load to Azure

In [ ]:
# using pyspark
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

#Create PySpark DataFrame from Pandas
cleaned_sahie = spark.createDataFrame(cleaned_sahie) 
cleaned_sahie.printSchema()
cleaned_sahie.show()

root
-- Geography_Category: string (nullable = true)
-- State_Name: string (nullable = true)
-- County_Name: string (nullable = true)
-- Age_Category: string (nullable = true)
-- Race_Category: string (nullable = true)
-- Sex_Category: string (nullable = true)
-- Income_Category: string (nullable = true)
-- Percent_of_Demographic_Uninsured_by_Income_Category: double (nullable = true)
-- Percent_of_Demographic_Insured_by_Income_Category: double (nullable = true)
-- Total_Percent_of_Demographic_Uninsured: double (nullable = true)
-- Total_Percent_of_Demographic_Insured: double (nullable = true)

+--------------------+----------+-----------+--------------+--------------------+------------+--------------------+---------------------------------------------------+-------------------------------------------------+--------------------------------------+------------------------------------+
 Geography_Category|State_Name|County_Name| Age_Category| Race_Category|Sex_Category| Income_Category|Percent_of_Demographic_Uninsured_by_Income_Category|Percent_of_Demographic_Insured_by_Income_Category|Total_Percent_of_Demographic_Uninsured|Total_Percent_of_Demographic_Insured|
+--------------------+----------+-----------+--------------+--------------------+------------+--------------------+---------------------------------------------------+-------------------------------------------------+--------------------------------------+------------------------------------+
State geographic ...| Alabama| |Under 65 years| All races| both sexes| All income levels| 11.6| 88.4| 11.6| 88.4|
State geographic ...| Alabama| |Under 65 years| All races| both sexes|At or below 200% ...| 18.9| 81.1| 6.8| 29.0|
State geographic ...| Alabama| |Under 65 years| All races| both sexes|At or below 250% ...| 18.0| 82.0| 8.0| 36.3|
State geographic ...| Alabama| |Under 65 years| All races| both sexes|At or below 138% ...| 19.9| 80.1| 4.8| 19.1|
State geographic ...| Alabama| |Under 65 years| All races| both sexes|At or below 400% ...| 15.1| 84.9| 10.1| 56.7|
State geographic ...| Alabama| |Under 65 years| All races| both sexes|Between 138% - 40...| 12.5| 87.5| 5.3| 37.5|
State geographic ...| Alabama| |Under 65 years| All races| male| All income levels| 12.9| 87.1| 12.9| 87.1|
State geographic ...| Alabama| |Under 65 years| All races| male|At or below 200% ...| 20.8| 79.2| 7.0| 26.5|
State geographic ...| Alabama| |Under 65 years| All races| male|At or below 250% ...| 20.0| 80.0| 8.4| 33.7|
State geographic ...| Alabama| |Under 65 years| All races| male|At or below 138% ...| 21.6| 78.4| 4.7| 17.1|
State geographic ...| Alabama| |Under 65 years| All races| male|At or below 400% ...| 16.9| 83.1| 11.0| 54.2|
State geographic ...| Alabama| |Under 65 years| All races| male|Between 138% - 40...| 14.5| 85.5| 6.3| 37.1|
State geographic ...| Alabama| |Under 65 years| All races| female| All income levels| 10.4| 89.6| 10.4| 89.6|
State geographic ...| Alabama| |Under 65 years| All races| female|At or below 200% ...| 17.3| 82.7| 6.6| 31.4|
State geographic ...| Alabama| |Under 65 years| All races| female|At or below 250% ...| 16.3| 83.7| 7.6| 38.9|
State geographic ...| Alabama| |Under 65 years| All races| female|At or below 138% ...| 18.5| 81.5| 4.8| 21.1|
State geographic ...| Alabama| |Under 65 years| All races| female|At or below 400% ...| 13.5| 86.5| 9.2| 59.1|
State geographic ...| Alabama| |Under 65 years| All races| female|Between 138% - 40...| 10.5| 89.5| 4.4| 38.0|
State geographic ...| Alabama| |Under 65 years|Non-Hispanic whit...| both sexes| All income levels| 10.0| 90.0| 10.0| 90.0|
State geographic ...| Alabama| |Under 65 years|Non-Hispanic whit...| both sexes|At or below 200% ...| 19.5| 80.5| 5.4| 22.2|
+--------------------+----------+-----------+--------------+--------------------+------------+--------------------+---------------------------------------------------+-------------------------------------------------+--------------------------------------+----------------

In [ ]:
# Mount the capstone container
storageAccount = "gen10datafund2205"
storageContainer = "capstone-group2-data"
clientSecret = "-ZS8Q~NwOKfwEpVOg3Teb1pPtxDbz616XjlXLbuU"
clientid = "2ca50102-5717-4373-b796-39d06568588d"
mount_point = "/mnt/capstone-group2-data/dataout"
    
    
configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs)

/mnt/capstone-group2-data/dataout has been unmounted.
Out[7]: True

In [ ]:
cleaned_sahie.write.mode("overwrite").option("header", "true").csv("/mnt/capstone-group2-data/dataout/cleandata/cleanSAHIE")

## Load to SQL

In [ ]:
# Mount the capstone container
storageAccount = "gen10datafund2205"
storageContainer = "capstone-group2-data"
clientSecret = "-ZS8Q~NwOKfwEpVOg3Teb1pPtxDbz616XjlXLbuU"
clientid = "2ca50102-5717-4373-b796-39d06568588d"
mount_point = "/mnt/capstone-group2-data/datain"
    
    
configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs)

/mnt/capstone-group2-data/datain has been unmounted.
Out[9]: True

In [ ]:
# variables for sql db loading
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"
database = "capstoneGroup2Database"
user = ""
password = ""

In [ ]:
# reading clean data
cleaned_sahie = spark.read.option("header", "true").csv("/mnt/capstone-group2-data/datain/cleandata/cleanSAHIE")
display(cleaned_sahie)

Geography_Category,State_Name,County_Name,Age_Category,Race_Category,Sex_Category,Income_Category,Percent_of_Demographic_Uninsured_by_Income_Category,Percent_of_Demographic_Insured_by_Income_Category,Total_Percent_of_Demographic_Uninsured,Total_Percent_of_Demographic_Insured
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,both sexes,At or below 138% of poverty,23.3,76.7,3.5,11.7
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,both sexes,At or below 400% of poverty,14.7,85.3,7.8,45.2
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,both sexes,Between 138% - 400% of poverty,11.3,88.7,4.2,33.5
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,male,All income levels,10.0,90.0,10.0,90.0
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,male,At or below 200% of poverty,22.2,77.8,4.9,17.1
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,male,At or below 250% of poverty,20.4,79.6,6.1,23.9
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,male,At or below 138% of poverty,24.9,75.1,3.8,11.4
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,male,At or below 400% of poverty,15.2,84.8,8.3,46.2
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,male,Between 138% - 400% of poverty,11.6,88.4,4.5,34.8
County geographic identifier,Missouri,Ralls County,50 to 64 years,All races,female,All income levels,8.9,91.1,8.9,91.1


#### Creating breakout tables

In [ ]:
# geocat breakout table
geocat_df = cleaned_sahie.select('Geography_Category').distinct()
geocat_df = geocat_df.withColumnRenamed('Geography_Category', 'GeoCategory')

display(geocat_df)

GeoCategory
State geographic identifier
County geographic identifier


In [ ]:
# geocat breakout sql load
table = "dbo.Geocat"

geocat_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
# age breakout table
age_df = cleaned_sahie.select('Age_Category').distinct()
age_df = age_df.withColumnRenamed('Age_Category', 'AgeLabel')

display(age_df)

AgeLabel
21 to 64 years
18 to 64 years
Under 65 years
50 to 64 years
40 to 64 years
Under 19 years


In [ ]:
# age breakout sql load
table = "dbo.Age"

age_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
# appending both sexes option onto sql sex database
bothsex_df = spark.createDataFrame([
    Row(SexLabel='both sexes'),
])

bothsex_df.show()

+----------+
 SexLabel|
+----------+
both sexes|
+----------+

In [ ]:
# sex breakout sql load
table = "dbo.Sex"

bothsex_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
# county breakout table
county_df = cleaned_sahie.select('County_Name').distinct()
county_df = county_df.withColumnRenamed('County_Name', 'CountyLabel')

display(county_df)

CountyLabel
Lewis and Clark County
Rock County
Canadian County
Webster County
Cumberland County
Woodward County
Rosebud County
Dakota County
Bryan County
Lycoming County


In [ ]:
# county breakout sql load
table = "dbo.County"

county_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
# income breakout table
income_df = cleaned_sahie.select('Income_Category').distinct()
income_df = income_df.withColumnRenamed('Income_Category', 'IncomeLabel')

display(income_df)

IncomeLabel
All income levels
At or below 138% of poverty
At or below 250% of poverty
Between 138% - 400% of poverty
At or below 400% of poverty
At or below 200% of poverty


In [ ]:
# income breakout sql load
table = "dbo.Income"

income_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

#### Joining tables

In [ ]:
table = "dbo.Geocat"

#Read from server
geocat_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
geocat_df.show()

+-----+--------------------+
GeoID| GeoCategory|
+-----+--------------------+
 9|State geographic ...|
 10|County geographic...|
+-----+--------------------+

In [ ]:
table = "dbo.Age"

#Read from server
age_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
age_df.show()

+-----+--------+
AgeID|AgeLabel|
+-----+--------+
 1| 18|
 2| 19|
 3| 20|
 4| 21|
 5| 22|
 6| 23|
 7| 24|
 8| 25|
 9| 26|
 10| 27|
 11| 28|
 12| 29|
 13| 30|
 14| 31|
 15| 32|
 16| 33|
 17| 34|
 18| 35|
 19| 36|
 20| 37|
+-----+--------+
only showing top 20 rows

In [ ]:
table = "dbo.Sex"

#Read from server
sex_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
sex_df.show()

+-----+----------+
SexID| SexLabel|
+-----+----------+
 1| female|
 2| male|
 3|both sexes|
+-----+----------+

In [ ]:
table = "dbo.County"

#Read from server
county_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
county_df.show()

+--------+--------------------+
CountyID| CountyLabel|
+--------+--------------------+
 3755|Lewis and Clark C...|
 3756| Rock County|
 3757| Canadian County|
 3758| Webster County|
 3759| Cumberland County|
 3760| Woodward County|
 3761| Rosebud County|
 3762| Dakota County|
 3763| Bryan County|
 3764| Lycoming County|
 3765| Bladen County|
 3766| Lucas County|
 3767| Garvin County|
 3768| Benton County|
 3769| Custer County|
 3770| Dundy County|
 3771| Knox County|
 3772| Morrill County|
 3773| Bernalillo County|
 3774| Clermont County|
+--------+--------------------+
only showing top 20 rows

In [ ]:
table = "dbo.Income"

#Read from server
income_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
income_df.show()

+--------+--------------------+
IncomeID| IncomeLabel|
+--------+--------------------+
 1| All income levels|
 2|At or below 138% ...|
 3|At or below 250% ...|
 4|Between 138% - 40...|
 5|At or below 400% ...|
 6|At or below 200% ...|
+--------+--------------------+

In [ ]:
table = "dbo.State"

#Read from server
state_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

+-------+--------------------+
StateID| StateLabel|
+-------+--------------------+
 1| Alabama|
 2| Alaska|
 3| Arizona|
 4| Arkansas|
 5| California|
 6| Colorado|
 7| Connecticut|
 8| Delaware|
 9|District of Columbia|
 10| Florida|
 11| Georgia|
 12| Hawaii|
 13| Idaho|
 14| Illinois|
 15| Indiana|
 16| Iowa|
 17| Kansas|
 18| Kentucky|
 19| Louisiana|
 20| Maine|
+-------+--------------------+
only showing top 20 rows

In [ ]:
table = "dbo.Race"

#Read from server
race_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

+------+--------------------+
RaceID| RaceLabel|
+------+--------------------+
 1| All races|
 2|Hispanic or Latin...|
 3|Non-Hispanic Asia...|
 4|Non-Hispanic blac...|
 5|Non-Hispanic whit...|
 6|Non-Hispanic, oth...|
+------+--------------------+

In [ ]:
cleaned_sahie = cleaned_sahie.join(geocat_df, cleaned_sahie.Geography_Category == geocat_df.GeoCategory, "left")
cleaned_sahie = cleaned_sahie.join(state_df, cleaned_sahie.State_Name == state_df.StateLabel, "left")
cleaned_sahie = cleaned_sahie.join(county_df, cleaned_sahie.County_Name == county_df.CountyLabel, "left")
cleaned_sahie = cleaned_sahie.join(race_df, cleaned_sahie.Race_Category == race_df.RaceLabel, "left")
cleaned_sahie = cleaned_sahie.join(age_df, cleaned_sahie.Age_Category == age_df.AgeLabel, "left")
cleaned_sahie = cleaned_sahie.join(sex_df, cleaned_sahie.Sex_Category == sex_df.SexLabel, "left")
cleaned_sahie = cleaned_sahie.join(income_df, cleaned_sahie.Income_Category == income_df.IncomeLabel, "left")

intermediate_sahie = cleaned_sahie.drop("GeoCategory","StateLabel","CountyLabel","RaceLabel","AgeLabel","SexLabel","IncomeLabel",
                                  "Geography_Category","State_Name","County_Name","Age_Category","Race_Category","Sex_Category","Income_Category")

display(intermediate_sahie)

Percent_of_Demographic_Uninsured_by_Income_Category,Percent_of_Demographic_Insured_by_Income_Category,Total_Percent_of_Demographic_Uninsured,Total_Percent_of_Demographic_Insured,GeoID,StateID,CountyID,RaceID,AgeID,SexID,IncomeID
23.3,76.7,3.5,11.7,10,26,3881,1,61,3,2
14.7,85.3,7.8,45.2,10,26,3881,1,61,3,5
11.3,88.7,4.2,33.5,10,26,3881,1,61,3,4
10.0,90.0,10.0,90.0,10,26,3881,1,61,2,1
22.2,77.8,4.9,17.1,10,26,3881,1,61,2,6
20.4,79.6,6.1,23.9,10,26,3881,1,61,2,3
24.9,75.1,3.8,11.4,10,26,3881,1,61,2,2
15.2,84.8,8.3,46.2,10,26,3881,1,61,2,5
11.6,88.4,4.5,34.8,10,26,3881,1,61,2,4
8.9,91.1,8.9,91.1,10,26,3881,1,61,1,1


In [ ]:
sql_sahie = intermediate_sahie \
  .withColumn("Percent_of_Demographic_Uninsured_by_Income_Category" ,
              cleaned_sahie["Percent_of_Demographic_Uninsured_by_Income_Category"]
              .cast(DecimalType(10,3)))   \
  .withColumn("Percent_of_Demographic_Insured_by_Income_Category",
              cleaned_sahie["Percent_of_Demographic_Insured_by_Income_Category"]
              .cast(DecimalType(10,3)))    \
  .withColumn("Total_Percent_of_Demographic_Uninsured"  ,
              cleaned_sahie["Total_Percent_of_Demographic_Uninsured"]
              .cast(DecimalType(10,3))) \
  .withColumn("Total_Percent_of_Demographic_Insured"  ,
              cleaned_sahie["Total_Percent_of_Demographic_Insured"]
              .cast(DecimalType(10,3))) \

sql_sahie.dtypes

Out[103]: [('Percent_of_Demographic_Uninsured_by_Income_Category', 'decimal(10,3)'),
 ('Percent_of_Demographic_Insured_by_Income_Category', 'decimal(10,3)'),
 ('Total_Percent_of_Demographic_Uninsured', 'decimal(10,3)'),
 ('Total_Percent_of_Demographic_Insured', 'decimal(10,3)'),
 ('GeoID', 'int'),
 ('StateID', 'int'),
 ('CountyID', 'int'),
 ('RaceID', 'int'),
 ('AgeID', 'int'),
 ('SexID', 'int'),
 ('IncomeID', 'int')]

In [ ]:
#Load to SQL Database
table = "dbo.SAHIE"

sql_sahie.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()